# Import necessary packages

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import os

import DiadFit as pf
pf.__version__


'1.0.5'

# Setup the folder access and paths

In [14]:
if os.name=='posix':
    slash='/'
else:
    slash='\\'

MasterFolder=os.path.dirname(os.path.dirname(os.getcwd()))

# #Folder to save figures

# figpath=MasterFolder +slash+'Figs'
# if not os.path.exists(figpath):
#     os.mkdir(figpath)

# #Folder to save full datasets

compilation_folder=MasterFolder +slash+"Data_processing_notebooks"+slash+'Data_processing_Uwekahuna'+slash+'Notebook_Compiled_data'
if not os.path.exists(compilation_folder):
    os.mkdir(compilation_folder)

# This compiles all the Raman data into one file, and exports

## First create the function

In [15]:

## This function compiles raman sessions found in the master folder (even in subdirectories of the specified folder)

def compile_Raman_sessions(*, path=None, sheet_name='Sheet1', keyword='Raman_session'):
    df = pd.DataFrame()

    for root, dirs, files in os.walk(path):
        for file in files:
            if keyword in file and file.endswith('.xlsx'):
                print(file)
                file_path = os.path.join(root, file)
                sheets = pd.read_excel(file_path, sheet_name)
                df = pd.concat([df, sheets], axis=0, ignore_index=True)
                unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
                df = df.drop(columns=unnamed_columns)

    return df

## Now run the function

In [16]:
UW = compile_Raman_sessions(path=MasterFolder+slash+"Data"+slash+"Data_Uwekahuna"+slash+"Raman",sheet_name='Sheet1',keyword='Raman_session')

Raman_session_December 12, 2022_fitted_2025-01-24.xlsx
Raman_session_November 3, 2022_fitted_2025-01-24.xlsx
Raman_session_December 13, 2022_fitted_2025-01-24.xlsx
Raman_session_November 8, 2022_fitted_2025-01-24.xlsx
Raman_session_November 11, 2022_fitted_2025-01-24.xlsx


# Now let's tidy names and filter bad analyses before exporting

In [17]:
# filter out bad ones. Including one standard that drifted. 
UW = UW[(~UW['filename'].str.contains('test|LL'))&(UW['Density g/cm3']>0)&~(UW['σ Density g/cm3']>0.04)]

UW['date_object']=pd.to_datetime(UW['date'])

UW=UW.sort_values(by=['date_object', 'filename'])


# def transform_name(name):
#     # Check if the name starts with 'KL0919'
#     if name.startswith('KL0919'):
#         # Replace 'KL0919' with '919' and replace '-' with '_'
#         new_name = '919' + name[6:].replace('-', '_')
#     else:
#         # Replace '-' with '_'
#         new_name = name.replace('-', '_')
#     return new_name
# # Apply the transformation to the column
# UW['consistent_name'] = UW['filename'].apply(transform_name)

UW=UW.reset_index(drop=True)

# UW['filename_4merge'] = UW['filename'] + '_index' + UW.index.astype(str)
# UW['consistent_name_4merge'] = UW['consistent_name'] + '_index' + UW.index.astype(str)

UW.to_clipboard(excel=True, index=False)

UW.to_excel(compilation_folder+'/'+"UW_Raman_allreps.xlsx")

UW


,filename,Density g/cm3,σ Density g/cm3,σ Density g/cm3 (from Ne+peakfit),σ Density g/cm3 (from densimeter),Corrected_Splitting,Corrected_Splitting_σ,Corrected_Splitting_σ_Ne,Corrected_Splitting_σ_peak_fit,power (mW),...,preferred_values,lower_values,upper_values,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,SO2_Diad_Ratio,SO2_mol_ratio,date_object
0,UW7-OX1-mX1-FI#1,0.104006,0.004466,0.002333,0.003808,103.012737,0.005889,0.000614,0.005869,12.028,...,0.997901,0.997895,0.997907,1150.858408,70.370487,43.109258,Spline,0.045794,0.019402,2022-11-03
1,UW7-OX1-mX1-FI#2,0.129867,0.004581,0.002487,0.003847,103.078003,0.006278,0.000616,0.006261,12.021,...,0.997898,0.997892,0.997904,1150.833954,106.861788,75.285717,Spline,0.044751,0.018969,2022-11-03
2,UW7-OX1-mX2-FI#1,0.091930,0.004120,0.001590,0.003801,102.982260,0.004012,0.000618,0.003973,12.039,...,0.997895,0.997889,0.997901,1150.858408,189.930369,134.247921,Spline,0.093753,0.038931,2022-11-03
3,UW7-OX2-FI#1,0.136645,0.003938,0.000770,0.003862,103.095109,0.001945,0.000621,0.001847,6.031,...,0.997891,0.997885,0.997897,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03
4,UW7-OX2-FI#2,0.144382,0.003958,0.000769,0.003883,103.114636,0.001941,0.000622,0.001842,6.018,...,0.997890,0.997884,0.997896,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,UW7-651-MI#1,0.115726,0.005712,0.004245,0.003821,103.042317,0.010714,0.001461,0.010641,12.038,...,0.997397,0.997383,0.997411,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-13
101,UW7-659-FI#1,0.185895,0.009735,0.009443,0.002370,103.208723,0.023175,0.001489,0.023188,12.073,...,0.997377,0.997362,0.997391,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-13
102,UW7-662-FI#1,0.121991,0.009112,0.008267,0.003831,103.058127,0.020864,0.001496,0.020865,12.122,...,0.997373,0.997359,0.997388,1150.687227,68.819112,39.991687,Spline,0.090984,0.037825,2022-12-13
103,UW7-662-FI#2,0.107391,0.007809,0.006816,0.003811,103.021281,0.017203,0.001501,0.017182,12.154,...,0.997371,0.997357,0.997386,1150.662773,110.166317,50.122234,Spline,0.251001,0.097840,2022-12-13


# UP TO HERE on Jan 24

In [6]:
rep_FI_names=pd.read_excel('Helper files/'+'Helperfile_UW.xlsx',sheet_name='Sheet1')
UW_reps = pd.merge(UW, rep_FI_names, on=['filename', 'date_object','sec since midnight'])
UW_reps

,filename,Density g/cm3,σ Density g/cm3,σ Density g/cm3 (from Ne+peakfit),σ Density g/cm3 (from densimeter),Corrected_Splitting,Corrected_Splitting_σ,Corrected_Splitting_σ_Ne,Corrected_Splitting_σ_peak_fit,power (mW),...,FI#,Sample_crystal,Sample_crystal_region,FI_name,Name_on_SEM,Notes regarding SEM matching,EBSD_simple_name,EBSD_grainID,EBSDname+grainID,EBSD_comment


## Now let's take mean of repeated analyses

In [7]:
grouped = KD24_reps.groupby('FI_name')

# Separate numeric and non-numeric columns
numeric_cols = KD24_reps.select_dtypes(include='number')
non_numeric_cols = KD24_reps.select_dtypes(exclude='number')

# Aggregate numeric columns by mean and standard deviation
numeric_KD24_averaged_mean = grouped[numeric_cols.columns].mean()
numeric_KD24_averaged_std = grouped[numeric_cols.columns].std()

# Take the first instance for non-numeric columns
non_numeric_KD24_averaged = grouped[non_numeric_cols.columns].first()

# Concatenate KD24_averageds and reindex columns to the original order
KD24_averaged = pd.concat([numeric_KD24_averaged_mean, non_numeric_KD24_averaged], axis=1)
KD24_averaged = KD24_averaged.reindex(columns=KD24_reps.columns)

# Add standard deviation columns after reindexing
KD24_averaged = pd.concat([KD24_averaged, numeric_KD24_averaged_std.add_suffix('_Raman_STD')], axis=1)

# Reset index to get a DataFrame
KD24_averaged = KD24_averaged.reset_index(drop=True)

row_counts = grouped.size().reset_index(name='row_count')

# Add a new column "averaged?" based on the row count
row_counts['averaged?'] = 'No'
row_counts.loc[row_counts['row_count'] > 1, 'averaged?'] = 'Yes'

# Merge the row counts back to the aggregated DataFrame
KD24_averaged = pd.merge(KD24_averaged, row_counts[['FI_name', 'averaged?']], on='FI_name', how='left')

# If there are missing values (groups with a single row), fill them with 'No'
KD24_averaged['averaged?'] = KD24_averaged['averaged?'].fillna('No')

#exclude a weird null average spectrum
# KD24_averaged=KD24_averaged[~(KD24_averaged['filename'].str.contains("186-9-118-A"))]
KD24_averaged.to_excel(compilation_folder+'/'+"KD24_Raman_averaged.xlsx")

KD24_averaged


,filename,Density g/cm3,σ Density g/cm3,σ Density g/cm3 (from Ne+peakfit),σ Density g/cm3 (from densimeter),Corrected_Splitting,Corrected_Splitting_σ,Corrected_Splitting_σ_Ne,Corrected_Splitting_σ_peak_fit,power (mW),...,lower_values_Raman_STD,upper_values_Raman_STD,Mount_Raman_STD,Name_on_SEM_Raman_STD,Notes regarding SEM matching_Raman_STD,EBSD_simple_name_Raman_STD,EBSD_grainID_Raman_STD,EBSDname+grainID_Raman_STD,EBSD_comment_Raman_STD,averaged?
0,FG04-A1-4-end_r5,0.691934,0.003428,0.002393,0.002391,104.330131,0.004693,0.000555,0.004667,6.016000,...,6.701971e-08,9.615113e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
1,FG04-A1-4-start_r1,0.688678,0.004276,0.003481,0.002390,104.323735,0.006836,0.000534,0.006827,6.001667,...,3.127646e-06,3.182375e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
2,KD24_c001_a1_FIA_r1,0.086661,0.007504,0.006332,0.003802,102.968963,0.015979,0.000515,0.016004,9.912000,...,1.172850e-06,1.192637e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
3,KD24_c001_a1_FIB,0.062668,0.006605,0.005391,0.003817,102.908411,0.013605,0.000513,0.013625,9.855000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,KD24_c001_a1_FIC,0.101646,0.009129,0.008297,0.003806,103.006781,0.020940,0.000509,0.020980,9.836000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
5,KD24_c002_a1_FIA,0.095798,0.004789,0.002912,0.003803,102.992023,0.007348,0.000507,0.007347,9.848000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
6,KD24_c003_a1_FIA,0.104360,0.004583,0.002549,0.003808,103.013631,0.006434,0.000506,0.006428,9.860000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
7,KD24_c003_a1_FIB,0.095473,0.004505,0.002416,0.003803,102.991202,0.006096,0.000505,0.006089,9.859000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
8,KD24_c003_a1_FIC,0.107992,0.005762,0.004321,0.003812,103.022797,0.010906,0.000505,0.010918,9.858000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
9,KD24_c003_a2_FID,0.118881,0.004831,0.002949,0.003826,103.050278,0.007443,0.000505,0.007442,9.891000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
